Her göreve özel kullanılacak Simple Transformers modeli, kullanıcının modeli kendi kullanım durumlarına göre kolayca uyarlamasını sağlamak için tonlarca yapılandırma seçeneğiyle birlikte gelir.
"https://simpletransformers.ai/docs/usage/ "

In [1]:
!pip install simpletransformers

Gerekli kütüphane ve metrikler tanımlandı

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
import torch
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer,AdamW , BertForSequenceClassification
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import StratifiedKFold


import datetime
import random
import seaborn as sns
import numpy as np
import time
import matplotlib.pyplot as plt


from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

'dbmdz/bert-base-turkish-128k-uncased':
vocab size -> 128k.   
The final training corpus has a size of 35GB and 44,04,976,662 tokens.

Temizlenmiş verinin kategorik etiketleri bir data frame e kaydedilip numerik olarak değiştirildi. 

In [3]:
# Dataset
df = pd.read_csv('datasets/clean_data.csv')
df['label'] = LabelEncoder().fit_transform(df['label'])

önceden eğitilmiş sınıflandırıcılar download edildi . 

In [4]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import KFold
from transformers import logging
logging.set_verbosity_error()

tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-128k-uncased', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained('dbmdz/bert-base-turkish-128k-uncased', num_labels=2)
# https://simpletransformers.ai/docs/classification-specifics/#supported-model-types


In [5]:
# check GPU
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    device = torch.device("cuda") #Current GPU;tensörleri ve modelleri cpu'dan gpu'ya taşır. 
else:
    raise SystemError('GPU device not found')

GPU kullanıyor mu ? Memory size nedir ?

In [6]:
!nvidia-smi

clean datadaki her satır için encode edilmiş değerleri hesaplar.
input_ids ->verilen token pozisyonunun gerçek token içerip içermediğini veya sıfır dolgulu bir konum olup olmadığını gösterir.
attention_mask yığındaki örneklerin uzunlukları farklı olsa bile, transformatöre bir yığın göndermemize olanak tanır. 

0'ları dolgu jetonlarının konumlarına ve 1'leri gerçek jetonların konumlarına yerleştirerek hengi input_ids lerin dummy olduğunu belirler. 

#max_length=10 olsun . [101, 2026, 2171, 2003, 11754, 102, 0, 0, 0, 0], 101:[CLS]ve 102:[SEP]  olduğu ifadeye karşılık gelir. 


encode_plus-> encode-dan farkı tokenizer ve kelimeleri kullanarak bir string-i, bir dizi id e çevirir, modelin iki diziyi ayırt etmesine izin veren bir tür maskeleme bekler


In [7]:
input_ids = []
attention_masks = []
max_len = 60 

for text in df['clean_data']:
    encoded_dict = tokenizer.encode_plus(
                        str(text),                     
                        add_special_tokens = True,
                        max_length = max_len,      
                        pad_to_max_length = True,
                        return_attention_mask = True, 
                        return_tensors = 'pt',
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)#Verilen boyutta(0) tensör dizisini birleştirir .
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(df['label'])

print('Original: ', df['clean_data'][0])
print('Token IDs:', input_ids[0])
print(labels[0])

Stratified K fold kullanılarak farklı etiketlerin verideki dağılımları dikkate alınmış oldu, 10 a bölündü . 
Veri artık input_ids ve labels isimli tensörlerden oluşmakta. Tensör nesneleri ile oluşan data frame sınıflandırıcıya verildi. 

In [ ]:

skf = StratifiedKFold(n_splits=10)
results = []
train_data = pd.DataFrame(columns=['texts','labels'])
train_data['texts'] = df.clean_data
train_data['labels'] = df.label
for train_index, val_index in skf.split(train_data['texts'], train_data['labels']):
  # splitting Dataframe (dataset not included)
    train_df = train_data.iloc[train_index]
    val_df = train_data.iloc[val_index]
    # Defining Model
    model = ClassificationModel('bert', 'dbmdz/bert-base-turkish-128k-uncased', num_labels=4,
                                args={'reprocess_input_data': True, 'overwrite_output_dir': True, 
                                      'num_train_epochs': 5, "train_batch_size": 64 , 
                                      "output_dir": "bert_model", "learning_rate": 5e-5, 
                                      "load_best_model_at_end": True, "use_early_stopping":True, "early_stopping_delta": 0.01,
                                     "max_length": 60, "tokenizer_name": tokenizer }, use_cuda=True, random_state = 42)
  # train the model
    model.train_model(train_df)
  # validate the model 
    result, model_outputs, wrong_predictions = model.eval_model(val_df, acc=accuracy_score)
    print(f'Acc : {result["acc"]}')
  # append model score
    results.append(result["acc"])


print("results",results)
print(f"Mean-Precision: {sum(results) / len(results)}")


In [ ]:
#model.get_named_parameters()

In [ ]:
predictions = model_outputs.argmax(axis=1)
actuals = val_df.labels.values

In [ ]:
from sklearn.metrics import balanced_accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

print(classification_report(actuals, predictions, digits=3))
print(balanced_accuracy_score(actuals, predictions))
cm = confusion_matrix(actuals, predictions, labels=[0,1,2,3])
print(cm)
tn, fp, fn, tp = cm
print((tn, fp, fn, tp))


In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=[0,1,2,3])
disp.plot()